In [ ]:
%matplotlib inline

import json
Settings = json.load(open('../settings.txt'))
import matplotlib.pyplot as plt
import numpy as np
from os.path import join
from cselect import color as cs
import sys
sys.path.insert(0,'../')
sys.path.insert(0,'../samples')
from mvpose.data import shelf
from time import time

root = Settings['data_root']
root = join(root, 'pak')
tmp = Settings['tmp']

import mvpose.data.kth_football2 as kth
from mvpose import pose
from mvpose.settings import get_settings
from paf_loader import Loader
from mvpose.evaluation import pcp

# =====================================
params = get_settings(
    scale_to_mm=1000,
    ms_radius=50,
    ms_between_distance=50
)
loader = Loader()
# =====================================

import mvpose.data.skeleton_augmentation as ska
model_path = '../data/model_poseprediction.h5'
gen = ska.LimbGenerator(model_path, params.scale_to_mm)

    
def generate_pcp_score(frame):
    global Settings, params
    alpha = 0.5
    
    Im, Y, Calib = shelf.get(root, frame)
    heatmaps, pafs = loader.load_confidence_map_and_paf(
        'shelf', Im, frame, dir=Settings['tmp'])
    detections = pose.estimate(Calib, heatmaps, pafs, 
                               settings=params, debug=False)
    #detections = gen.apply(detections)
    Humans = kth.transform3d_from_mscoco(detections)
    
    L_Arms = []
    U_Arms = []
    L_Legs = []
    U_Legs = []
    GTIDs = []
  
    for gtid, gt in enumerate(Y):
        if gt is None:
            continue
        
        larms = 0
        uarms = 0
        llegs = 0
        ulegs = 0
        avg = 0
        for d in Humans:
            r = pcp.evaluate(gt, d, alpha)
            larms_ = r.lower_arms
            uarms_ = r.upper_arms
            ulegs_ = r.upper_legs
            llegs_ = r.lower_legs
            avg_ = (larms_ + uarms_ + ulegs_ + llegs_) / 4
            if avg_ > avg:
                avg = avg_
                larms = larms_
                uarms = uarms_
                llegs = llegs_
                ulegs = ulegs_
        
        L_Arms.append(larms)
        U_Arms.append(uarms)
        L_Legs.append(llegs)
        U_Legs.append(ulegs)
        GTIDs.append(gtid)
    
    del heatmaps
    del pafs
    del detections
    del Im
    del Y
    del Calib
    
    return L_Arms, U_Arms, L_Legs, U_Legs, GTIDs

# =====================================

valid_frames = list(range(300, 600))
PER_GTID = {}
FRAMES = []
AVG = []

for frame in valid_frames:
    _start = time()
    L_Arms, U_Arms, L_Legs, U_Legs, GTIDs =\
        generate_pcp_score(frame)
    _end = time()
    print('frame ' + str(frame) + ', elapsed:', _end - _start)
    if len(L_Arms) > 0:
        for gtid, larms, uarms, llegs, ulegs in zip(
            GTIDs, L_Arms, U_Arms, L_Legs, U_Legs
        ):
            if not gtid in PER_GTID:
                PER_GTID[gtid] = {
                    'larms': [],
                    'uarms': [],
                    'llegs': [],
                    'ulegs': [],
                    'frame': []
                }
            PER_GTID[gtid]['larms'].append(larms)
            PER_GTID[gtid]['uarms'].append(uarms)
            PER_GTID[gtid]['llegs'].append(llegs)
            PER_GTID[gtid]['ulegs'].append(ulegs)
            PER_GTID[gtid]['frame'].append(frame)
            
            FRAMES.append(frame)
            AVG.append((larms + uarms + llegs + ulegs)/4)
            
            

/home/tanke/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


frame 300, elapsed: 3.364633560180664
elapsed: 3.194270372390747
elapsed: 1.4422030448913574
elapsed: 1.419602870941162
elapsed: 1.4513819217681885
elapsed: 1.4503655433654785
frame 301, elapsed: 12.811112642288208
elapsed: 1.4689021110534668
elapsed: 1.447533130645752
elapsed: 1.4878904819488525
elapsed: 1.44203782081604
elapsed: 1.476301670074463
frame 302, elapsed: 11.772243738174438
elapsed: 1.475175142288208
elapsed: 1.533588171005249
elapsed: 1.4539296627044678
elapsed: 1.4254639148712158
elapsed: 1.4310948848724365
frame 303, elapsed: 11.424134731292725
elapsed: 1.4259889125823975
elapsed: 1.44868803024292
elapsed: 1.4420440196990967
elapsed: 1.4348855018615723
elapsed: 1.4434077739715576
frame 304, elapsed: 12.147969961166382
elapsed: 1.454761028289795
elapsed: 1.443162441253662
elapsed: 1.4335594177246094
elapsed: 1.434183120727539
elapsed: 1.4307515621185303
frame 305, elapsed: 11.350787162780762
elapsed: 1.430459976196289
elapsed: 1.4247016906738281
elapsed: 1.43696093559265

elapsed: 1.4816524982452393
elapsed: 1.4979944229125977
elapsed: 1.5327188968658447
elapsed: 1.6078174114227295
elapsed: 1.5482008457183838
frame 347, elapsed: 12.01071310043335
elapsed: 1.493121862411499
elapsed: 1.547051191329956
elapsed: 1.5013477802276611
elapsed: 1.4874932765960693
elapsed: 1.4953112602233887
frame 348, elapsed: 13.230857372283936
elapsed: 1.501413345336914
elapsed: 1.490922212600708
elapsed: 1.470587968826294
elapsed: 1.4637808799743652
elapsed: 1.4939289093017578
frame 349, elapsed: 11.849987745285034
elapsed: 1.4386630058288574
elapsed: 1.4717621803283691
elapsed: 1.4492456912994385
elapsed: 1.451763391494751
elapsed: 1.4727389812469482
frame 350, elapsed: 12.791734218597412
elapsed: 1.4349606037139893
elapsed: 1.449305772781372
elapsed: 1.4667763710021973
elapsed: 1.4682621955871582
elapsed: 1.441615343093872
frame 351, elapsed: 11.681252002716064
elapsed: 1.4282636642456055
elapsed: 1.5371193885803223
elapsed: 1.5399751663208008
elapsed: 1.5587797164916992
el

elapsed: 1.564878225326538
elapsed: 1.6751015186309814
elapsed: 1.6890435218811035
frame 393, elapsed: 12.361473321914673
elapsed: 1.6968116760253906
elapsed: 1.6996116638183594
elapsed: 1.6053714752197266
elapsed: 1.5461456775665283
elapsed: 1.5606350898742676
frame 394, elapsed: 13.85584044456482
elapsed: 1.4728548526763916
elapsed: 1.5216748714447021
elapsed: 1.5301921367645264
elapsed: 1.5222420692443848
elapsed: 1.5799980163574219
frame 395, elapsed: 11.818723917007446
elapsed: 1.5010390281677246
elapsed: 1.5517473220825195
elapsed: 1.538916826248169
elapsed: 1.5063095092773438
elapsed: 1.5543339252471924
frame 396, elapsed: 13.568018913269043
elapsed: 1.4977517127990723
elapsed: 1.4996941089630127
elapsed: 1.4733209609985352
elapsed: 1.4883842468261719
elapsed: 1.5320723056793213
frame 397, elapsed: 11.899132251739502
elapsed: 1.4770419597625732
elapsed: 1.5238115787506104
elapsed: 1.5122675895690918
elapsed: 1.5049879550933838
elapsed: 1.487654209136963
frame 398, elapsed: 13.51

elapsed: 1.4772467613220215
elapsed: 1.4691078662872314
elapsed: 1.4672422409057617
frame 440, elapsed: 11.653880834579468
elapsed: 1.4947621822357178
elapsed: 1.460866928100586
elapsed: 1.4482908248901367
elapsed: 1.453336238861084
elapsed: 1.4782192707061768
frame 441, elapsed: 13.343790292739868
elapsed: 1.559248447418213
elapsed: 1.547715425491333
elapsed: 1.5481736660003662
elapsed: 1.524918794631958
elapsed: 1.496626615524292
frame 442, elapsed: 11.914765357971191
elapsed: 1.4717745780944824
elapsed: 1.4649534225463867
elapsed: 1.444751262664795
elapsed: 1.4593939781188965
elapsed: 1.467085599899292
frame 443, elapsed: 13.195204734802246
elapsed: 1.4694769382476807
elapsed: 1.4660305976867676
elapsed: 1.4512765407562256
elapsed: 1.450678825378418
elapsed: 1.4526429176330566
frame 444, elapsed: 11.76986837387085
elapsed: 1.5786032676696777
elapsed: 1.5947418212890625
elapsed: 1.562619924545288
elapsed: 1.5230655670166016
elapsed: 1.5318756103515625
frame 445, elapsed: 13.557122468

elapsed: 1.4631762504577637
frame 486, elapsed: 11.919301271438599
elapsed: 1.5523803234100342
elapsed: 1.654242992401123
elapsed: 1.6244540214538574
elapsed: 1.577282428741455
elapsed: 1.5749914646148682
frame 487, elapsed: 13.693786859512329
elapsed: 1.5438532829284668
elapsed: 1.57423996925354
elapsed: 1.5375051498413086
elapsed: 1.5391356945037842
elapsed: 1.5147731304168701
frame 488, elapsed: 12.151271104812622
elapsed: 1.4909162521362305
elapsed: 1.4725799560546875
elapsed: 1.4562478065490723
elapsed: 1.4353013038635254
elapsed: 1.452822208404541
frame 489, elapsed: 13.20398235321045
elapsed: 1.4420382976531982
elapsed: 1.453587532043457
elapsed: 1.433377981185913
elapsed: 1.4383952617645264
elapsed: 1.428386926651001
frame 490, elapsed: 11.746894121170044
elapsed: 1.4662160873413086
elapsed: 1.4951269626617432
elapsed: 1.4798369407653809
elapsed: 1.474010944366455
elapsed: 1.5028164386749268
frame 491, elapsed: 16.042402267456055
elapsed: 1.4754958152770996
elapsed: 1.495729446

elapsed: 1.6906991004943848
elapsed: 1.7129371166229248
elapsed: 1.6412274837493896
elapsed: 1.6284456253051758
elapsed: 1.6136271953582764
frame 533, elapsed: 22.961360454559326
elapsed: 1.6542890071868896
elapsed: 1.6385560035705566
elapsed: 1.626180648803711
elapsed: 1.5797481536865234
elapsed: 1.571312665939331
frame 534, elapsed: 18.348206520080566
elapsed: 1.6985876560211182
elapsed: 1.671726942062378
elapsed: 1.6338505744934082
elapsed: 1.6066222190856934
elapsed: 1.5747146606445312
frame 535, elapsed: 13.681612491607666
elapsed: 1.5763635635375977
elapsed: 1.6031379699707031
elapsed: 1.5658957958221436
elapsed: 1.5207350254058838
elapsed: 1.5491573810577393
frame 536, elapsed: 33.09557914733887
elapsed: 1.6770970821380615
elapsed: 1.6557962894439697
elapsed: 1.5778417587280273
elapsed: 1.5681910514831543
elapsed: 1.5502312183380127
frame 537, elapsed: 32.791743755340576
elapsed: 1.6213421821594238
elapsed: 1.6236977577209473
elapsed: 1.5698060989379883
elapsed: 1.56197953224182

In [ ]:
total_avg = []
for key, values in PER_GTID.items():
    print('actor ', key)
    print('\tuarms:', np.mean(values['uarms']))
    print('\tlarms:', np.mean(values['larms']))
    print('\tulegs:', np.mean(values['ulegs']))
    print('\tllegs:', np.mean(values['llegs']))
    avg = np.mean([
        np.mean(values['uarms']),
        np.mean(values['larms']),
        np.mean(values['ulegs']),
        np.mean(values['llegs'])
    ])
    total_avg.append(avg)
    print('\tavg:  ', avg)
print('\navg*:  ', np.mean(total_avg))

In [ ]:
idx = 0

fig = plt.figure(figsize=(16,8))
ax = fig.add_subplot(111)
ax.plot(range(len(AVG)), AVG)
ax.set_ylim([0, 1.1])

ax.plot([idx, idx], [0, 1.5], alpha=0.5)
 
print("avg:\t", AVG[idx])
print('frame:\t', FRAMES[idx])
FRAME = FRAMES[idx]

plt.show()